# Approach A

In [6]:
identifier = 'bD_Z2_S2'
identifier_specify = identifier[-5:]
tex = f"{identifier}/output/calibro_report.tex"
_idfPath = f"../eplus_diagnosis/idf-Oct/G2/auto-cal/bD_alpha_{identifier_specify}.idf"

import sys
from pathlib import Path
from _approach_a import *
here = Path.cwd()
src_dir = next(p for p in [here, *here.parents] if (p / "src").exists()) / "src"
sys.path.insert(0, str(src_dir))
from ReIDF import ReIDF1

In [7]:
def generate_office_string(pattern, ending=';'):
    result = []
    for value, count in pattern:
        result.extend([value] * count)
    return ', '.join(result) + ending

# Defaults (model's baseline values if no calibrated value exists)
d1_htgsp_office_default = 18
d2_htgsp_office_st_default = 15

# Placeholder schedules (raw form prior to replacement)
d1_htg_updated_weekday = generate_office_string([
    ('15', 8), ('{{d1_htgsp_office}}', 11), ('15', 5)
])

d2_htg_updated_weekend = generate_office_string([('{{d2_htgsp_office_st}}', 24)])

d2_htg_updated_weekday = generate_office_string([
    ('{{d2_htgsp_office_st}}', 8), ('{{d1_htgsp_office}}', 11), ('{{d2_htgsp_office_st}}', 5)
])

# Original user-defined lists (must preserve format)
ReUnit_HVAC = [
    ('MaterialExt_W', 3, '{{a1_u_value_external_walls}}'),
    ('MaterialExt_Win', 2, '{{a3_u_value_external_windows}}'),
    ('MaterialExt_Win 1', 2, '{{a3_u_value_external_windows}}'),
    ('MaterialExt_Win 2', 2, '{{a3_u_value_external_windows}}'),
    ('MaterialExt_Win', 3, '{{a4_g_value}}'),
    ('MaterialExt_Win 1', 3, '{{a4_g_value}}'),
    ('MaterialExt_Win 2', 3, '{{a4_g_value}}'),
    ('htg_sch_office_br__Weekday', 3, d1_htg_updated_weekday),
    ('htg_sch_office_br__Weekday', 3, d2_htg_updated_weekday),
    ('htg_sch_office_br__Sunday', 3, d2_htg_updated_weekend),
]

ReClass_HVAC = [
    ('ZoneVentilation:WindandStackOpenArea', 3, '{{e1_natural_ventilation_rate}}'),
    ('ZoneVentilation:WindandStackOpenArea', 7, '{{e1_natural_ventilation_rate}}'),
    ('ZoneInfiltration:DesignFlowRate', 7, '{{a5_infiltration_rate}}'),
    ('Fan:ConstantVolume', 3, '{{b1_ahu_fan_efficiency}}'),
    ('AirLoopHVAC', 11, '{{b10_airloophvac}};'),
    ('SetpointManager:OutdoorAirReset', 3, '{{b12_sat}}'),
]


In [8]:
df, param_map, ReUnit_CalibroMID, ReClass_CalibroMID = process_all(
    tex,
    ReUnit_HVAC,
    ReClass_HVAC,
    drop_missing=True 
)
ReClass_CalibroMID, ReUnit_CalibroMID

([('ZoneVentilation:WindandStackOpenArea', 3, '1.223'),
  ('ZoneVentilation:WindandStackOpenArea', 7, '1.223'),
  ('ZoneInfiltration:DesignFlowRate', 7, '0.872'),
  ('SetpointManager:OutdoorAirReset', 3, '23.360')],
 [('htg_sch_office_br__Weekday',
   3,
   '15, 15, 15, 15, 15, 15, 15, 15, 23.779, 23.779, 23.779, 23.779, 23.779, 23.779, 23.779, 23.779, 23.779, 23.779, 23.779, 15, 15, 15, 15, 15;'),
  ('htg_sch_office_br__Weekday',
   3,
   '17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 23.779, 23.779, 23.779, 23.779, 23.779, 23.779, 23.779, 23.779, 23.779, 23.779, 23.779, 17.775, 17.775, 17.775, 17.775, 17.775;'),
  ('htg_sch_office_br__Sunday',
   3,
   '17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775;')])

In [9]:
ReClass_CalibroMID

[('ZoneVentilation:WindandStackOpenArea', 3, '1.223'),
 ('ZoneVentilation:WindandStackOpenArea', 7, '1.223'),
 ('ZoneInfiltration:DesignFlowRate', 7, '0.872'),
 ('SetpointManager:OutdoorAirReset', 3, '23.360')]

In [10]:
_idfPath = Path(_idfPath)
updated = ReIDF1(oriPath=_idfPath, 
                 ReClass=ReClass_CalibroMID, 
                 ReUnit=ReUnit_CalibroMID, 
                 cusPath=_idfPath.parent / ".." / "post-cal" / ( 'POST' + _idfPath.stem + _idfPath.suffix))

ReIDF: → /Users/rui.bo/Desktop/Working/1-phd_mainworks/Y3/eplus_diagnosis/idf-Oct/G2/post-cal/POSTbD_alpha_Z2_S2.idf
ReUnit: [{'htg_sch_office_br__Weekday': 1}, {'htg_sch_office_br__Weekday': 1}, {'htg_sch_office_br__Sunday': 1}]
ReClass: [{'ZoneVentilation:WindandStackOpenArea': 15}, {'ZoneVentilation:WindandStackOpenArea': 15}, {'ZoneInfiltration:DesignFlowRate': 14}, {'SetpointManager:OutdoorAirReset': 1}]


# Approach B

In [9]:
df, param_map, ReUnit_PLACEHOLDER, ReClass_PLACEHOLDER = process_all(
    tex,
    ReUnit_HVAC,
    ReClass_HVAC,
    drop_missing=True,
    overwrite_with_derived=False  # Keep original placeholders but drop unresolved ones
)
df

,PARAMETER,ESTIMATE,LOWER,UPPER
0,e1_natural_ventilation_rate,1.296,0.860,1.345
1,b10_airloophvac,0.770,0.644,1.026
2,b12_sat,22.903,17.028,26.015
3,d2_htgsp_office_st,15.969,15.047,17.851
4,d1_htgsp_office,23.692,21.187,23.972


In [ ]:
import re
import os
from pathlib import Path

import numpy as np
import pandas as pd
from scipy.stats import norm
from pyDOE2 import lhs
from SALib.sample.morris import sample as morris_sample
import matplotlib.pyplot as plt
import seaborn as sns

# =============================================================
# 0. DISTRIBUTION INFERENCE FROM df (LaTeX TABLE)
# =============================================================

# df is expected to contain columns: PARAMETER, ESTIMATE, LOWER, UPPER


def normalize_bound(b):
    """Convert 'NA' / None / numeric to a clean Python bound (or None)."""
    if b is None:
        return None
    if isinstance(b, str) and b.upper() == "NA":
        return None
    return float(b)


def detect_truncation(mu, low, up, param_name, user_bounds=None, eps=1e-6):
    """Decide if a parameter should be treated as truncated.

    Priority:
    1) user_bounds (if provided)
    2) automatic detection (touching 0 or 1)
    3) None (no truncation)
    """
    # 1. User-specified bounds
    if user_bounds is not None and param_name in user_bounds:
        lo, hi = user_bounds[param_name]
        lo = normalize_bound(lo)
        hi = normalize_bound(hi)
        if lo is not None or hi is not None:
            return (lo, hi)

    # 2. Automatic detection
    if abs(up - 1.0) < eps:
        return (0.0, 1.0)
    if abs(low - 0.0) < eps:
        return (0.0, None)

    return None


def compute_sigma_from_CI(mu, low, up):
    """Approximate sigma from a symmetric 95% CI, using the larger side."""
    dev = max(mu - low, up - mu)
    return dev / 1.96


def classify_with_truncation(mu, low, up, param_name, user_bounds=None):
    """Return (family, bounds) where family in {Normal, Lognormal, TruncatedNormal}."""
    trunc = detect_truncation(mu, low, up, param_name, user_bounds=user_bounds)
    if trunc is not None:
        return "TruncatedNormal", trunc

    # Normal vs Lognormal by asymmetry
    d_low = (mu - low) / mu
    d_up = (up - mu) / mu

    # avoid division by 0
    asym_ratio = abs(d_up - d_low) / max(d_low, d_up, 1e-9)

    if asym_ratio < 0.25:
        return "Normal", None
    else:
        return "Lognormal", None


def infer_distributions_with_trunc(df, user_bounds=None):
    """Infer distribution family + parameters for each row of df."""
    rows = []

    for _, row in df.iterrows():
        param = row["PARAMETER"]
        mu = float(row["ESTIMATE"])
        low = float(row["LOWER"])
        up = float(row["UPPER"])

        family, bounds = classify_with_truncation(mu, low, up, param_name=param, user_bounds=user_bounds)

        sigma = compute_sigma_from_CI(mu, low, up)
        cv = sigma / mu if mu != 0 else np.nan

        if family == "Normal":
            rows.append({
                "Parameter": param,
                "Distribution": "Normal",
                "Mean": mu,
                "Sigma": sigma,
                "CV": cv,
                "Lower": np.nan,
                "Upper": np.nan,
            })

        elif family == "Lognormal":
            sigma_ln2 = np.log(1 + cv ** 2)
            sigma_ln = np.sqrt(sigma_ln2)
            mu_ln = np.log(mu) - 0.5 * sigma_ln2
            rows.append({
                "Parameter": param,
                "Distribution": "Lognormal",
                "Mean": mu,
                "Mu_log": mu_ln,
                "Sigma_log": sigma_ln,
                "CV": cv,
                "Lower": np.nan,
                "Upper": np.nan,
            })

        elif family == "TruncatedNormal":
            lo, hi = bounds
            rows.append({
                "Parameter": param,
                "Distribution": "TruncatedNormal",
                "Mean": mu,
                "Sigma": sigma,
                "CV": cv,
                "Lower": lo,
                "Upper": hi,
            })

    return pd.DataFrame(rows)


# =============================================================
# 1. CONVERT INFERRED DIST TABLE → parameter_specs + variable_names
# =============================================================

# We keep the structure of your previous workflow as much as possible.
# parameter_specs will now support: normal, lognormal, truncated_normal.


def convert_inferred_to_specs(dist_df):
    """Convert dist_df (from infer_distributions_with_trunc) into
    parameter_specs and variable_names for sampling.
    """
    parameter_specs = []
    variable_names = []

    for _, row in dist_df.iterrows():
        name = row["Parameter"]
        dist = row["Distribution"].lower()
        variable_names.append(name)

        if dist == "normal":
            parameter_specs.append({
                "distribution": "normal",
                "mean": float(row["Mean"]),
                "std": float(row["Sigma"]),
            })

        elif dist == "lognormal":
            parameter_specs.append({
                "distribution": "lognormal",
                "mu_log": float(row["Mu_log"]),
                "sigma_log": float(row["Sigma_log"]),
            })

        elif dist == "truncatednormal":
            parameter_specs.append({
                "distribution": "truncated_normal",
                "mean": float(row["Mean"]),
                "std": float(row["Sigma"]),
                "lower": normalize_bound(row["Lower"]),
                "upper": normalize_bound(row["Upper"]),
            })

        else:
            raise ValueError(f"Unsupported inferred distribution: {row['Distribution']}")

    return parameter_specs, variable_names


# =============================================================
# 2. LHS + MORRIS SAMPLING (EXTENDED TO HANDLE NEW DISTRIBUTIONS)
# =============================================================


def generate_samples(method, num_samples, parameter_specs, random_seed=None, N=10, num_levels=10):
    if method == 'lhs':
        return generate_lhs_samples_mixed(num_samples, parameter_specs, random_seed)
    elif method == 'morris':
        return generate_morris_samples(num_samples, parameter_specs, N, num_levels)
    else:
        raise ValueError("Unsupported method. Use lhs or morris.")


def generate_lhs_samples_mixed(num_samples, parameter_specs, random_seed=None):
    """LHS sampling for a mixture of normal, lognormal, truncated_normal and uniform.

    This keeps your previous structure but extends the mapping.
    """
    if random_seed is not None:
        np.random.seed(random_seed)

    num_parameters = len(parameter_specs)
    lhs_samples = lhs(num_parameters, samples=num_samples)

    for i, spec in enumerate(parameter_specs):
        dist = spec['distribution']

        if dist == 'uniform':
            min_val, max_val = spec['range']
            lhs_samples[:, i] = np.round(min_val + lhs_samples[:, i] * (max_val - min_val), 6)

        elif dist == 'normal':
            mean, std = spec['mean'], spec['std']
            lhs_samples[:, i] = np.round(norm.ppf(lhs_samples[:, i], loc=mean, scale=std), 6)

        elif dist == 'lognormal':
            mu_log, sigma_log = spec['mu_log'], spec['sigma_log']
            lhs_samples[:, i] = np.round(
                np.exp(norm.ppf(lhs_samples[:, i], loc=mu_log, scale=sigma_log)), 6
            )

        elif dist == 'truncated_normal':
            mean, std = spec['mean'], spec['std']
            lower = spec.get('lower', None)
            upper = spec.get('upper', None)

            a = -np.inf if lower is None else (lower - mean) / std
            b = np.inf if upper is None else (upper - mean) / std

            Fa = norm.cdf(a)
            Fb = norm.cdf(b)
            u = lhs_samples[:, i]
            u_trunc = Fa + u * (Fb - Fa)
            lhs_samples[:, i] = np.round(norm.ppf(u_trunc, loc=mean, scale=std), 6)

        else:
            raise ValueError(f"Unsupported distribution in parameter_specs: {dist}")

    return np.nan_to_num(lhs_samples)


def generate_morris_samples(num_samples, parameter_specs, N=10, num_levels=10):
    """Morris sampling (still supports uniform + normal as before).

    For lognormal / truncated_normal you can either:
    - map through an underlying normal space, or
    - restrict Morris usage to parameters declared uniform/normal.

    Here we keep it simple and *only* support uniform + normal
    (lognormal / truncated_normal would raise a clear error).
    """
    num_parameters = len(parameter_specs)

    problem_definition = {
        'num_vars': num_parameters,
        'names': [f'param_{i}' for i in range(num_parameters)],
        'bounds': [[0, 1] for _ in range(num_parameters)]
    }

    morris_samples = morris_sample(problem=problem_definition, N=N, num_levels=num_levels)

    for i, spec in enumerate(parameter_specs):
        dist = spec['distribution']
        if dist == 'uniform':
            min_val, max_val = spec['range']
            morris_samples[:, i] = np.round(min_val + morris_samples[:, i] * (max_val - min_val), 6)
        elif dist == 'normal':
            mean = spec['mean']
            std = spec['std']
            p = 0.1 + 0.8 * morris_samples[:, i]
            morris_samples[:, i] = np.round(norm.ppf(p, loc=mean, scale=std), 6)
        else:
            raise ValueError(f"Morris currently supports only uniform/normal, got {dist} for index {i}")

    return morris_samples


def visualize_lhs_samples(lhs_samples, variable_names):
    lhs_df = pd.DataFrame(lhs_samples, columns=variable_names)
    sns.pairplot(lhs_df)
    plt.suptitle('Scatter Plots of LHS Samples', y=1.02)
    plt.show()


# =============================================================
# 3. IDF CREATION (UNCHANGED, REUSABLE)
# =============================================================


def create_idf_files(template_file_name, cPath, variable_names, values,
                     schedules, output_name_template, overwrite=False):
    if not overwrite:
        print("Overwrite disabled. Skipping IDF creation.")
        return []

    cPath = Path(cPath)
    cPath.mkdir(exist_ok=True)

    template_text = Path(template_file_name).read_text()
    file_paths = []

    for i, value_set in enumerate(values, start=0):
        f_out = template_text

        if i == 0:
            print("Processing 1st file ...")

        # Replace parameters: here we assume placeholders are {{ParameterName}}
        for var, val in zip(variable_names, value_set):
            placeholder = f"{{{{{var}}}}}"
            if placeholder in f_out:
                f_out = f_out.replace(placeholder, str(val))

        # Replace schedules (if provided)
        for schedule_name, schedule_list in (schedules or {}).items():
            placeholder = f"{{{{{schedule_name}_schedule}}}}"
            if placeholder in f_out:
                schedule_string = ", ".join(f"{v:.2f}" for v in schedule_list[i]) + ";"
                f_out = f_out.replace(placeholder, schedule_string)

        out_name = output_name_template.format(number=f"{i:04d}")
        out_file = cPath / f"{out_name}.idf"
        out_file.write_text(f_out)
        file_paths.append(out_file)

    return file_paths


# =============================================================
# 4. MAIN ENTRY (NO EXCEL, DIRECTLY FROM df + user_bounds)
# =============================================================


def main_from_latex_df(
    df,
    user_bounds,
    template_file_name,
    method='lhs',
    num_samples=10,
    overwrite=False,
    plot_lhs=False,
    output_name=None,
    schedules=None,
):
    """Unified workflow:

    1) infer distributions (Normal / Lognormal / TruncatedNormal) from df
    2) convert to parameter_specs + variable_names
    3) generate LHS / Morris samples
    4) create IDF files using {{Parameter}} placeholders

    No Excel / extra CSV is required.
    """

    # 1) Infer distributions
    dist_df = infer_distributions_with_trunc(df, user_bounds=user_bounds)
    print("==== Inferred distributions ====")
    print(dist_df)

    # 2) Convert to sampling specs
    parameter_specs, variable_names = convert_inferred_to_specs(dist_df)

    # 3) Generate samples
    samples = generate_samples(method, num_samples, parameter_specs)

    if plot_lhs and method == 'lhs':
        visualize_lhs_samples(samples, variable_names)

    # 4) Output folder: based on template location + output_name
    template_dir = Path(template_file_name).parent.resolve()

    if output_name is not None:
        calibration_group = template_dir / output_name
    else:
        calibration_group = template_dir / "samples_from_latex"

    calibration_group.mkdir(exist_ok=True)

    # 5) Create IDFs
    output_name_template = "sample_{number}"
    idf_paths = create_idf_files(
        template_file_name,
        calibration_group,
        variable_names,
        samples,
        schedules,
        output_name_template,
        overwrite,
    )

    # 6) Return samples and folder; no CSV autosave
    sample_df = pd.DataFrame(samples, columns=variable_names)
    return sample_df, calibration_group, dist_df


# =============================================================
# 5. EXAMPLE USAGE (SKETCH)
# =============================================================




In [ ]:

user_bounds = {
    'b10_airloophvac': (None, 1),   # upper bound manually set to 1
    'a5_infiltration_rate': (0, None),  # force lower bound at 0
}

template_file_name = "idf-Oct/G2/auto-cal/1bD_alpha_Z2_S2.idf"
output_name = "bD_Z2_S2_UQ"

sample_df, calibration_group, dist_df = main_from_latex_df(
    df=df,
    user_bounds=user_bounds,
    template_file_name=template_file_name,
    method='lhs',
    num_samples=10,
    overwrite=True,
    plot_lhs=True,
    output_name=output_name,
    schedules=None,
)





In [ ]:
# sample_df.to_csv(calibration_group / "samples.csv", index=False)

# Archive